In [1]:
import dash 
from dash import dcc, html, Input, Output, dash_table 
import plotly.express as px
import pandas as pd 
import json
import plotly.graph_objects as go 
import sys 
import base64
from numpy import random

sys.path.append('../')

import omdutils as omd

In [2]:
sql = 'SELECT * FROM cars'
db = '../SourceData/datasets.db'
df = omd.from_sqlite(sql, db) 
# Add a random "jitter" to model_year to spread out the plot
df['year'] = random.randint(-4,5,len(df))*0.10 + df['model_year']
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,year
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,70.1
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,70.1
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,69.7
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,70.2
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,69.9


## Version 1

In [12]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(
        id='mpg_scatter',
        figure={
            'data': [go.Scatter(
                x = df['year']+1900,  # our "jittered" data
                y = df['mpg'],
                text = df['name'],
                hoverinfo = 'text',
                mode = 'markers'
            )],
            'layout': go.Layout(
                title = 'mpg.csv dataset',
                xaxis = {'title': 'model year'},
                yaxis = {'title': 'miles per gallon'},
                hovermode='closest'
            )
        }
    )
])

if __name__ == '__main__':
    app.run_server()

## Version 2

In [9]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([   
        dcc.Graph(
            id='mpg_scatter',
            figure={
                'data': [go.Scatter(
                    x=df['year'] + 1900,  
                    y=df['mpg'],
                    text=df['name'],
                    hoverinfo='text',
                    mode='markers',
                    marker={'size': 10, 'opacity': 0.5}
                )],
                'layout': go.Layout(
                    title='mpg.csv dataset',
                    xaxis={'title': 'model year'},
                    yaxis={'title': 'miles per gallon'},
                    hovermode='closest'
                )
            }
        )
    ], style={'width': '50%', 'display': 'inline-block'}),
    html.Div([  
        dcc.Graph(
            id='mpg_line',
            figure={
                'data': [go.Scatter(
                    x=[0, 1],
                    y=[0, 1],
                    mode='lines'
                )],
                'layout': go.Layout(
                    title='acceleration',
                    margin={'l': 0}
                )
            }
        )
    ], style={'width': '20%', 'height': '50%', 'display': 'inline-block'})
])

@app.callback(
    Output('mpg_line', 'figure'),
    [Input('mpg_scatter', 'hoverData')]
)
def callback_graph(hoverData):
    if hoverData is None:
        return {
            'data': [go.Scatter(
                x=[0, 1],
                y=[0, 1],
                mode='lines'
            )],
            'layout': go.Layout(
                title='acceleration',
                margin={'l': 0}
            )
        }

    v_index = hoverData['points'][0]['pointIndex']
    fig = {
        'data': [go.Scatter(
            x=[0, 1],
            y=[0, 60 / df.iloc[v_index]['acceleration']],
            mode='lines',
            line={'width': 2 * df.iloc[v_index]['cylinders']}
        )],
        'layout': go.Layout(
            title=df.iloc[v_index]['name'],
            xaxis={'visible': False},
            yaxis={'visible': False, 'range': [0, 60 / df['acceleration'].min()]},
            margin={'l': 0},
            height=300
        )
    }
    return fig

if __name__ == '__main__':
    app.run_server()

## Version 3

In [13]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([   # this Div contains our scatter plot
    dcc.Graph(
        id='mpg_scatter',
        figure={
            'data': [go.Scatter(
                x = df['year']+1900,  # our "jittered" data
                y = df['mpg'],
                text = df['name'],
                hoverinfo = 'text',
                mode = 'markers',
                marker = {'size': 12}
            )],
            'layout': go.Layout(
                title = 'mpg.csv dataset',
                xaxis = {'title': 'model year'},
                yaxis = {'title': 'miles per gallon'},
                hovermode='closest'
            )
        }
    )], style={'width':'50%','display':'inline-block'}),
    html.Div([  # this Div contains our output graph and vehicle stats
    dcc.Graph(
        id='mpg_line',
        figure={
            'data': [go.Scatter(
                x = [0,1],
                y = [0,1],
                mode = 'lines'
            )],
            'layout': go.Layout(
                title = 'acceleration',
                margin = {'l':0}
            )
        }
    ),
    dcc.Markdown(
        id='mpg_stats'
    )
    ], style={'width':'20%', 'height':'50%','display':'inline-block'})
])

@app.callback(
    Output('mpg_line', 'figure'),
    [Input('mpg_scatter', 'hoverData')])
def callback_graph(hoverData):
    if hoverData is None:
        return {
            'data': [go.Scatter(
                x=[0, 1],
                y=[0, 1],
                mode='lines'
            )],
            'layout': go.Layout(
                title='acceleration',
                margin={'l': 0}
            )
        }
    v_index = hoverData['points'][0]['pointIndex']
    fig = {
        'data': [go.Scatter(
            x = [0,1],
            y = [0,60/df.iloc[v_index]['acceleration']],
            mode='lines',
            line={'width':2*df.iloc[v_index]['cylinders']}
        )],
        'layout': go.Layout(
            title = df.iloc[v_index]['name'],
            xaxis = {'visible':False},
            yaxis = {'visible':False, 'range':[0,60/df['acceleration'].min()]},
            margin = {'l':0},
            height = 300
        )
    }
    return fig

@app.callback(
    Output('mpg_stats', 'children'),
    [Input('mpg_scatter', 'hoverData')])
def callback_stats(hoverData):
    v_index = hoverData['points'][0]['pointIndex']
    stats = """
        {} cylinders
        {}cc displacement
        0 to 60mph in {} seconds
        """.format(df.iloc[v_index]['cylinders'],
            df.iloc[v_index]['displacement'],
            df.iloc[v_index]['acceleration'])
    return stats

if __name__ == '__main__':
    app.run_server()

[2024-08-24 12:42:55,955] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/oliverdreger/opt/anaconda3/envs/ml2024v3/lib/python3.10/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/oliverdreger/opt/anaconda3/envs/ml2024v3/lib/python3.10/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/oliverdreger/opt/anaconda3/envs/ml2024v3/lib/python3.10/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/oliverdreger/opt/anaconda3/envs/ml2024v3/lib/python3.10/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/Users/oliverdreger/opt/anaconda3/envs/ml2024v3/lib/python3.10/site-packages/dash/dash.py", line 1373, in dispatch
    ctx.run